## APP PASSWORD

In [1]:
APP_PASSWORD = 'hksx zxjb wztw ncvp'

## Ebay

In [5]:
import imaplib
import email
from email.header import decode_header

# IMAP server settings for Gmail
IMAP_SERVER = 'imap.gmail.com'
USERNAME = 'dateng2016@gmail.com'
PASSWORD = APP_PASSWORD

# Connect to the IMAP server
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(USERNAME, PASSWORD)

# Select the mailbox (e.g., 'INBOX')
mail.select('INBOX')

# Search for emails based on specific criteria (e.g., subject, sender, date)
search_criteria = '(SUBJECT "Da, your order is confirmed")'
result, data = mail.search(None, search_criteria)

# Get the email IDs
email_ids = data[0].split()

# Retrieve the most recent email (assuming the search returns emails sorted by date)
latest_email_id = email_ids[-1]

# Fetch the email data (headers and body)
result, email_data = mail.fetch(latest_email_id, '(RFC822)')
raw_email = email_data[0][1]

# Parse the raw email data
msg = email.message_from_bytes(raw_email)

# Extract email information (e.g., sender, subject)
sender = msg['From']
subject = decode_header(msg['Subject'])[0][0]

# Print email information
print("Sender:", sender)
print("Subject:", subject)

# You can access email body and attachments similarly
# body = msg.get_payload()

# Function to get the email body
def get_email_body(msg):
    if msg.is_multipart():
        # If the message is multipart, iterate over each part
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == "text/plain" or content_type == "text/html":
                # If the part is plain text or HTML, return its payload
                return part.get_payload(decode=True).decode()
    else:
        # If the message is not multipart, return its payload
        return msg.get_payload(decode=True).decode()

# Example usage:
email_body = get_email_body(msg)
# print("Email Body:", email_body)


# Close the connection
mail.close()
mail.logout()

Sender: eBay <ebay@ebay.com>
Subject: Da, your order is confirmed


('BYE', [b'LOGOUT Requested'])

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(email_body, 'html.parser')

name_tag = soup.find('a', class_='title')
price_tags = soup.find_all('td', class_="item")
print(f'Merchant name: {sender.split(" ")[0]}')
print()
print('Item: ', name_tag.string)
print('Price: ',price_tags[-1].string)
print('Date: ', msg['Date'])

Merchant name: eBay

Item:  
3.5mm Jack To 2 Rca Stereo Audio Cable Adapter 6ft 10...

Price:  
$5.07

Date:  Tue, 13 Feb 2024 17:33:11 -0700


## Gmail API


In [73]:
import os.path
import os
from bs4 import BeautifulSoup

# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode

# for dealing with attachement MIME types
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
FOLDER_NAME = 'ReceiptFolder'



def start_service():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
    # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        results = service.users().labels().list(userId="me").execute()
        labels = results.get("labels", [])

        if not labels:
            print("No labels found.")

        # print("Labels:")
        for label in labels:
        #   print(label["name"])
            pass
        return service

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

def parse_parts(parts, file_count, folder_name=FOLDER_NAME):
    """
    Utility function that parses the content of an email partition
    """

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    if parts:
        for part in parts:
            filename = part.get("filename")
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")

            binary_data = urlsafe_b64decode(data)
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                file_count[0] += 1
                parse_parts(part.get("parts"), folder_name, file_count)
            if mimeType == "text/html":
                # if the email part is an HTML content
                # save the HTML file and optionally open it in the browser
                if not filename:
                    file_count[0] += 1
                    filename = "index"+str(file_count[0])+".html"
                filepath = os.path.join(folder_name, filename)
                print("Saving HTML to", filepath)
                with open(filepath, "wb") as f:
                    f.write(binary_data)
                # print(binary_data.decode())
                return binary_data.decode()

        
def get_ebay_results(service, msg_id, file_count):
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    html_content = parse_parts(msg['payload']['parts'], file_count, 'ReceiptFolder')
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'].lower() == 'date':
            date = header['value']
        if header['name'].lower() == 'from':
            sender = header['value']
    
    sender = sender.split(' ')[0]
        
    soup = BeautifulSoup(html_content, 'html.parser')
    name_tag = soup.find('a', class_='title')
    name = name_tag.string
    price_tag = soup.find('p', class_='labelValueValue').find('b')
    price = price_tag.string

    return sender, date, name, price
    
def get_walmart_results(service, msg_id, file_count, folder_name = FOLDER_NAME):
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    binary_data = urlsafe_b64decode(msg['payload']['body']['data'])
    file_count[0] += 1
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    filename = "index"+str(file_count[0])+".html"
    filepath = os.path.join(folder_name, filename)
    print("Saving HTML to", filepath)
    with open(filepath, "wb") as f:
        f.write(binary_data)
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'].lower() == 'date':
            date = header['value']
        if header['name'].lower() == 'from':
            sender = header['value']
    sender = sender.split('"')[1].split('.')[0]


    return sender, date

def get_amazon_results(service, msg_id, file_count, folder_name = FOLDER_NAME):
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    html_content = parse_parts(msg['payload']['parts'], file_count, folder_name)
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'].lower() == 'date':
            date = header['value']
        if header['name'].lower() == 'from':
            sender = header['value']
    sender = sender.split('"')[1].split('.')[0]
    return sender, date
    


if __name__ == "__main__":
    service = start_service()
    ebay_search_query = 'from:ebay@ebay.com subject:"Da, your order is confirmed"'
    walmart_search_query = 'from:help@walmart.com subject:"Da, thanks for your order"'
    amazon_search_query = 'from:auto-confirm@amazon.com'

    query = amazon_search_query

    # Call the Gmail API to search for messages
    messages = service.users().messages().list(userId='me', q=query).execute()
    html_list = []

    # Check if any messages match the search criteria
    file_count = [0]
    if 'messages' in messages:
        for message in messages['messages']:
            # Get the message details
            msg_id = message['id']
            # For eBay
            if query == ebay_search_query:
                sender, date, name, price = get_ebay_results(service, msg_id, file_count)
                print('Merchant Name: ', sender)
                print('Date: ', date)
                print('Item Name: ', name)
                print("Price: ", price)
            # For Walmart
            if query == walmart_search_query:
                sender, date = get_walmart_results(service, msg_id, file_count)
                print('Merchant Name: ', sender)
                print('Date: ', date)


            # For amazon
            if query == amazon_search_query:
                sender, date = get_amazon_results(service, msg_id, file_count)
                print('Merchant Name: ', sender)
                print('Date: ', date)
            
            # Avoid having too many queries
            if file_count[0] >= 10:
                break


    else:
        print('No messages found matching the search criteria.')


    # Parse the date string into a datetime object
    # date = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S %z')


Saving HTML to ReceiptFolder/index1.html
Amazon
Thu, 8 Feb 2024 00:01:07 +0000
Saving HTML to ReceiptFolder/index2.html
Amazon
Wed, 7 Feb 2024 19:10:36 +0000
Saving HTML to ReceiptFolder/index3.html
Amazon
Tue, 19 Dec 2023 23:56:35 +0000
Saving HTML to ReceiptFolder/index4.html
Amazon
Fri, 20 Oct 2023 01:25:00 +0000
Saving HTML to ReceiptFolder/index5.html
Amazon
Fri, 13 Oct 2023 18:39:33 +0000
Saving HTML to ReceiptFolder/index6.html
Amazon
Mon, 21 Aug 2023 02:03:10 +0000
Saving HTML to ReceiptFolder/index7.html
Amazon
Mon, 21 Aug 2023 01:58:30 +0000
Saving HTML to ReceiptFolder/index8.html
Amazon
Wed, 3 May 2023 00:03:55 +0000
Saving HTML to ReceiptFolder/index9.html
Amazon
Fri, 28 Apr 2023 00:25:00 +0000
Saving HTML to ReceiptFolder/index10.html
Amazon
Thu, 2 Mar 2023 00:27:31 +0000


## SCRATCH

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_list[0], 'html.parser')

name_tag = soup.find('a', class_='title')
price_tags = soup.find_all('td', class_="item")
# print(f'Merchant name: {sender.split(" ")[0]}')
print()
print('Item: ', name_tag.string)
print('Price: ',price_tags[-1].string)
# print('Date: ', msg['Date'])


Item:  
3.5mm Jack To 2 Rca Stereo Audio Cable Adapter 6ft 10...

Price:  
$5.07



In [16]:
with open('ReceiptFolder/index1.html', 'r', encoding='utf-8') as file:
    html_string = file.read()

soup = BeautifulSoup(html_string,'html.parser')
price_tag = soup.find('p', class_='labelValueValue').find('b')
print(price_tag.string)

$4.66
